# Swine Flu and Seasonal Flu Vaccination Prediction Project

-----------------------------------------------------------------------------------------

## 1. BUSINESS UNDERSTANDING

## Stakeholder: Center for Disease Control (CDC) or Department of Health and Human Services(DHHS)

## Task: What is the likelihood of an individual getting vaccinated based upon survey data? Can we use these predictions for actionable policy choices?


Dataset curated from the National 2009 H1N1 Flu Survey, conducted by the CDC in 2009-2010. Obtained from:
https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/



## 2. DATA CLEANING

### A detailed overview and explanation of our data cleaning steps:

The following steps were taken to clean our database.

First, we dropped the columns of employment_industry and employment_occupation. Both columns showed a high number of null datapoints with no good way to impute data. Rather than lose 13330 rows of data, we decided to lose these two columns and let the rest of our socio-economic data stand in as proxy.

Next, we faced categories with a scattering of data with no real way to impute information. We simply dropped rows with nulls in these columns, slightly (but only slightly) reducing our data set.

Then, we created a few special case imputations for some of our data. Most of these columns relate to responses to survery data. Here we imputed default answers as made sense. For instance, if someone fails to answer for whether or not they have a chronic medical condition, we assume that they do not. If a condition was severe enough to be chronic, we feel that a respondent would be aware of that enough to mark yes. These assumptions won't be right all the time, but we believe that they'll still add to the predictive power of the model. 

The decisions we made for imputations are as follows:

* Blanks on survey questions were imputed to the 'neutral' or 'don't know' values.

* In behavioral columns we imputed with the majority class. 

* We assumed that if a patient had a doctor's recommendation they would say so.

* And finally was assumed that if a respondent had close contact with a child under 6 months they would know that, and we imputed a 'no' reseponse.

After this data cleaning we are left with a large number of null values in the health insurance column. We felt that this factor was too important to drop, as we had been forced to for employment_industry, for instance. Instead we decided to impute that data via KNN imputation. However, that will wait until after we've split the dataset into test and training data.

**Load packages**

In [1]:

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer, KNNImputer 

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve

**Loading Data**

In [2]:
features_df = pd.read_csv('Data/training_set_features.csv', index_col="respondent_id")
labels_df = pd.read_csv('Data/training_set_labels.csv', index_col="respondent_id")
df = features_df.join(labels_df, how = 'inner')
# check that the rows between the features and the labels match up
# np.testing.assert_array_equal(features_df.index.values, labels_df.index.values)

# # merge features_df and labels_df 
# df = labels_df.merge(features_df, how = 'inner', on='respondent_id')

# # drop duplicate 
# df.drop_duplicates(inplace=True)

In [3]:
df.shape

(26707, 37)

In [4]:
df.info()

# 17  health_insurance, 36  employment_industry, 37  employment_occupation  about 50% missing 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

**Check for null values**

In [5]:
df.isna().sum()

h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

**Functions to clean data and drop columns**

In [6]:
def basicdropna(dataframe, column_list):
    dataframe.dropna(subset=column_list, inplace=True)

In [7]:
general_dropna = ['health_worker', 'education','income_poverty', 'marital_status', 
                    'rent_or_own', 'employment_status', 'household_adults', 
                    'household_children' ]
basicdropna(df, general_dropna)

In [8]:
def columndrop(dataframe, column_list):
    dataframe.drop(column_list, axis = 1, inplace=True)

In [9]:
drop_columns =  ['employment_industry',  'employment_occupation', 'hhs_geo_region']


columndrop(df, drop_columns)

**Can we use the median value to imput missing values?**

In [10]:
df.median()

h1n1_concern                   2.0
h1n1_knowledge                 1.0
behavioral_antiviral_meds      0.0
behavioral_avoidance           1.0
behavioral_face_mask           0.0
behavioral_wash_hands          1.0
behavioral_large_gatherings    0.0
behavioral_outside_home        0.0
behavioral_touch_face          1.0
doctor_recc_h1n1               0.0
doctor_recc_seasonal           0.0
chronic_med_condition          0.0
child_under_6_months           0.0
health_worker                  0.0
health_insurance               1.0
opinion_h1n1_vacc_effective    4.0
opinion_h1n1_risk              2.0
opinion_h1n1_sick_from_vacc    2.0
opinion_seas_vacc_effective    4.0
opinion_seas_risk              2.0
opinion_seas_sick_from_vacc    2.0
household_adults               1.0
household_children             0.0
h1n1_vaccine                   0.0
seasonal_vaccine               0.0
dtype: float64

**Insight: for survey items where the scoring is 1-5, use 3 (don't know) to fill missing values. We cannot simply use the median, as this might indicate inaccurate knowledge on the part of the respondant.**


In [11]:
survey_col = ['opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective', 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc']

In [12]:
def impute_missing_data(dataframe, column_list, fillvalue):
    '''column_list can be a single column or a list of columns'''
    for column in column_list:
        dataframe[column].fillna(fillvalue, inplace = True)

In [13]:
impute_missing_data(df, survey_col, 3)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21863 entries, 0 to 26706
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 21825 non-null  float64
 1   h1n1_knowledge               21792 non-null  float64
 2   behavioral_antiviral_meds    21815 non-null  float64
 3   behavioral_avoidance         21720 non-null  float64
 4   behavioral_face_mask         21853 non-null  float64
 5   behavioral_wash_hands        21842 non-null  float64
 6   behavioral_large_gatherings  21805 non-null  float64
 7   behavioral_outside_home      21817 non-null  float64
 8   behavioral_touch_face        21774 non-null  float64
 9   doctor_recc_h1n1             20253 non-null  float64
 10  doctor_recc_seasonal         20253 non-null  float64
 11  chronic_med_condition        21701 non-null  float64
 12  child_under_6_months         21863 non-null  float64
 13  health_worker   

In [15]:
x_df = df.copy()

survey_col = ['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective', 'opinion_seas_risk','opinion_seas_sick_from_vacc']

behavior_col = ['behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask',
                'behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home','behavioral_touch_face']

doc_rec = ['doctor_recc_h1n1','doctor_recc_seasonal']

# def impute_missing_data(dataframe, column_list, fillvalue):
#     for column in column_list:
#         dataframe[column].fillna(fillvalue, inplace = True)
      
    
impute_missing_data(x_df, survey_col, 3)
impute_missing_data(x_df, ['h1n1_concern'], 2)
impute_missing_data(x_df, ['h1n1_knowledge'], 0)
impute_missing_data(x_df, behavior_col, 0)
impute_missing_data(x_df, doc_rec, 0)
impute_missing_data(x_df, ['chronic_med_condition'], 0)
impute_missing_data(x_df, ['child_under_6_months'], 0)

In [16]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21863 entries, 0 to 26706
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 21863 non-null  float64
 1   h1n1_knowledge               21863 non-null  float64
 2   behavioral_antiviral_meds    21863 non-null  float64
 3   behavioral_avoidance         21863 non-null  float64
 4   behavioral_face_mask         21863 non-null  float64
 5   behavioral_wash_hands        21863 non-null  float64
 6   behavioral_large_gatherings  21863 non-null  float64
 7   behavioral_outside_home      21863 non-null  float64
 8   behavioral_touch_face        21863 non-null  float64
 9   doctor_recc_h1n1             21863 non-null  float64
 10  doctor_recc_seasonal         21863 non-null  float64
 11  chronic_med_condition        21863 non-null  float64
 12  child_under_6_months         21863 non-null  float64
 13  health_worker   

**Insight: health insurance is missing many values and is usually an imporant indicator for treatment. We will use knnimputer to fill in missing values in order to keep the feature.**

In [17]:
X=x_df.drop(['h1n1_vaccine','seasonal_vaccine'], axis=1)
y=x_df[['h1n1_vaccine','seasonal_vaccine']]

# Train test split, do this before OHE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**OHE for object data types, then impute with KNNImputer**

In [18]:
# create OHE for objects, do this before imputer

cat_col_list = [i for i in X_train.select_dtypes(include='object').columns]

nb_list_for_ohe = ['h1n1_concern', 'h1n1_knowledge', 'opinion_h1n1_vacc_effective',
'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
'opinion_seas_risk', 'opinion_seas_sick_from_vacc']

# Fits OHE on a subset of columns, then reintegrates them into the
# Origional dataframe. Do this after initial cleaning, before 
# health insurace imputation.

ohe = OneHotEncoder(drop='first', sparse=False)

def fit_trans_ohe(X_dataframe, columns):
    dums = ohe.fit_transform(X_dataframe[columns])
    dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_dataframe.index)
    df_cols_dropped = X_dataframe.drop(columns, axis = 1)
    dums_df_concated = pd.concat([df_cols_dropped, dums_df], axis=1)
    return dums_df_concated

#We should end up with a fitted ohe instance called 'ohe'

In [19]:
X_train_ohe = fit_trans_ohe(X_train, cat_col_list+nb_list_for_ohe)

**Fitting an imputer for Health Insurance using socio-economic features, then pulling from a dataframe that has already been OneHotEncoded**

In [20]:
    
socio_economic_column_list = ["x0_35 - 44 Years","x0_45 - 54 Years","x0_55 - 64 Years","x0_65+ Years",
                              "x1_< 12 Years","x1_College Graduate","x1_Some College","x2_Hispanic",
                              "x2_Other or Multiple","x2_White","x3_Male", "x4_> $75,000", "x4_Below Poverty",
                              "x5_Not Married", "x6_Rent", "x7_Not in Labor Force","x7_Unemployed",
                              "x8_MSA, Principle City",'x8_Non-MSA', 'health_insurance']

# Fitting an imputer for Health Insurance using socio-economic features, 
# pulling from a dataframe that has already been OneHotEncoded


soc_eco_h_i_imputer_knn = KNNImputer()

def soc_eco_KNN_imputer(imputer, dataframe, column_list):
    soc_econ_base = dataframe[column_list]
    soc_econ_imputed = pd.DataFrame(imputer.fit_transform(soc_econ_base), 
                                         columns = soc_econ_base.columns,
                                        index=soc_econ_base.index)
    remainder_df = dataframe.drop(column_list, axis = 1)
    output_df = remainder_df.join(soc_econ_imputed)
    output_df.health_insurance = output_df.health_insurance.round() 

    return output_df


In [21]:
X_train_imputed = soc_eco_KNN_imputer(soc_eco_h_i_imputer_knn, X_train_ohe, socio_economic_column_list)

**Now we OHE for the test set. Takes X test dataframe and list of columns to encoded**

In [22]:

def trans_ohe(X_dataframe, columns):
    dums = ohe.transform(X_dataframe[columns])
    dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_dataframe.index)
    df_cols_dropped = X_dataframe.drop(columns, axis = 1)
    dums_df_concated = pd.concat([df_cols_dropped, dums_df], axis=1)
    return dums_df_concated

In [23]:
X_test_ohe = trans_ohe(X_test, cat_col_list+nb_list_for_ohe)

In [24]:
def imputer_transform_only(imputer, dataframe, column_list):
    soc_econ_base = dataframe[column_list]
    soc_econ_imputed = pd.DataFrame(imputer.transform(soc_econ_base), 
                                         columns = soc_econ_base.columns,
                                        index=soc_econ_base.index)
    remainder_df = dataframe.drop(column_list, axis = 1)
    output_df = remainder_df.join(soc_econ_imputed)
    output_df.health_insurance = output_df.health_insurance.round()
    
    return output_df

In [25]:
X_test_imputed = imputer_transform_only(soc_eco_h_i_imputer_knn, X_test_ohe, socio_economic_column_list)

### We now have a working dataset of: 
    'X_train_imputed' and 'y_train' to fit models to, 'X_test_imputed' to generate predictions, and 'y_test' to validate models with.

**Save training and test data to csv files for model creation and tuning**

In [26]:
# import os
# cwd = os.getcwd()
# path = cwd + '/Data/X_train_imputed.csv'
# X_train_imputed.to_csv(path, index=False)

In [27]:

# cwd = os.getcwd()
# path = cwd + '/Data/X_test_imputed.csv'
# X_test_imputed.to_csv(path, index=False)

In [28]:

# cwd = os.getcwd()
# path = cwd + '/Data/y_train.csv'
# y_train.to_csv(path, index=False)

In [29]:

# cwd = os.getcwd()
# path = cwd + '/Data/y_test.csv'
# y_test.to_csv(path, index=False)

## 3. MODEL CREATION AND TUNING 

### 3.0 Baseline Model

In [30]:
from sklearn.dummy import DummyClassifier

# H1N1
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train_imputed, y_train.h1n1_vaccine)

# Seasonal Flu
dummy_clf_2 = DummyClassifier(strategy="most_frequent")
dummy_clf_2.fit(X_train_imputed, y_train.seasonal_vaccine)


print('Accuracy Score of Baseline Model (H1N1)', round(dummy_clf.score(X_train_imputed, y_train.h1n1_vaccine),4)) 
print('Accuracy Score of Baseline Model(Seasonal Flu)', round(dummy_clf_2.score(X_train_imputed, y_train.seasonal_vaccine),4)) 


Accuracy Score of Baseline Model (H1N1) 0.7835
Accuracy Score of Baseline Model(Seasonal Flu) 0.5319


### 3.1 kNN
#### Model Creation: kNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier()
knn.fit(X_train_imputed, y_train)

KNeighborsClassifier()

In [32]:
print(cross_val_score(knn, X_train_imputed, y_train, scoring='roc_auc', cv=5))
print(cross_val_score(knn, X_train_imputed, y_train, scoring='roc_auc', cv=5).mean())

[0.76306193 0.75290382 0.75306046 0.74841586 0.75484394]


KeyboardInterrupt: 

#### Model Tuning for KNN using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
#for h1n1

grid = {'n_neighbors': list(range(1, 29, 2)),
       'metric': ['euclidean', 'manhattan', 'minkowski']}

knn_grid_search = GridSearchCV(knn, grid, cv=3)
knn_grid_search.fit(X_train_imputed, y_train['h1n1_vaccine'])

In [ ]:
#for seasonal

grid = {'n_neighbors': list(range(1, 29, 2)),
       'metric': ['euclidean', 'manhattan', 'minkowski']}

knn_grid_search2 = GridSearchCV(knn, grid, cv=3)
knn_grid_search2.fit(X_train_imputed, y_train['seasonal_vaccine'])

#### Best parameters for KNN models

In [ ]:
#h1n1 best parameters
knn_grid_search.best_params_

In [ ]:
#seasonal best parameters

knn_grid_search2.best_params_
    

#### Models with optimized parameters and mean cross val scores

In [ ]:
knn_f_h1 = KNeighborsClassifier(n_neighbors=19, metric='manhattan')
knn_f_h1.fit(X_train_imputed, y_train)

print(cross_val_score(knn_f_h1, X_test_imputed, y_test['h1n1_vaccine'], scoring='roc_auc', cv=5).mean())

In [ ]:
knn_f_s = KNeighborsClassifier(n_neighbors=27, metric='manhattan')
knn_f_s.fit(X_train_imputed, y_train)

print(cross_val_score(knn_f_s, X_test_imputed, y_test['seasonal_vaccine'], scoring='roc_auc', cv=5).mean())

#### Predict probabilities and plot their distribution

In [ ]:

y_pred_h1n1 = knn_f_h1.predict_proba(X_test_imputed)

sns.distplot(y_pred_h1n1);

In [ ]:
y_pred_s = knn_f_s.predict_proba(X_test_imputed)

sns.distplot(y_pred_s);

## KNN has moderately good performance. Will try DecisionTreeClassifier to see if score improves.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train_imputed, y_train)

In [ ]:
print(cross_val_score(dt, X_train_imputed, y_train['seasonal_vaccine'], scoring='roc_auc', cv=5).mean())
print(cross_val_score(dt, X_train_imputed, y_train['h1n1_vaccine'], scoring='roc_auc', cv=5).mean())

### 3.2 Decision Tree Classifier
#### GridSearchCV for DecisionTreeClassifier

In [ ]:
#seasonal
dt_grid = {'criterion':['gini', 'entropy'],
          'splitter': ['best', 'random']}

dt_gridsearch_s = GridSearchCV(dt, dt_grid, cv=3)
dt_gridsearch_s.fit(X_train_imputed, y_train['seasonal_vaccine'])

In [ ]:
#h1n1

dt_gridsearch_h1 = GridSearchCV(dt, dt_grid, cv=3)
dt_gridsearch_h1.fit(X_train_imputed, y_train['h1n1_vaccine'])

#### Best parameters for DecisionTreeClassifier

In [ ]:
dt_gridsearch_s.best_params_

In [ ]:
dt_gridsearch_h1.best_params_

#### Cross validation for DecisionTree model

In [ ]:
dt_seasonal = DecisionTreeClassifier(criterion='entropy', splitter='random')

dt_seasonal.fit(X_train_imputed, y_train)

print(cross_val_score(dt_seasonal, X_train_imputed, y_train['seasonal_vaccine'], scoring='roc_auc', cv=5).mean())

In [ ]:
#since the best parameters are the same, we can score the H1N1 with the same parameters
dt_h1n1 = DecisionTreeClassifier(criterion='entropy', splitter='random')

dt_h1n1.fit(X_train_imputed, y_train)


print(cross_val_score(dt_h1n1, X_train_imputed, y_train['h1n1_vaccine'], scoring='roc_auc', cv=5).mean())

### Insight: DecisionTreeClassifier is not a good model for this data. kNN is obviously a better performer. Will attempt LogisticRegression to see if there is an improvement.

### 3.3 Logistic Regression

#### Logistic Regression with default parametor setting 
##### Insight: ROC_AUC scores for both H1N1 model and Seasonal Flu model are higher than Decision Tree and kNN. ROC_AUC score is stable via cross-validation checks; this model has a lower risk of overfitting. 

In [33]:
lr = LogisticRegression()

# H1N1 
h1n1 = cross_val_score(estimator=lr, X=X_train_imputed, y=y_train.h1n1_vaccine, 
                cv=5, scoring='roc_auc')


# Seasonal
sea = cross_val_score(estimator=lr, X=X_train_imputed, y=y_train.seasonal_vaccine, 
                cv=5, scoring='roc_auc')

print(f'ROC_AUC cv scores for H1N1 model:{h1n1}')
print(f'ROC_AUC cv scores for Seasonal:{sea}')

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

ROC_AUC cv scores for H1N1 model:[0.83848129 0.83708961 0.85211532 0.82365835 0.84492364]
ROC_AUC cv scores for Seasonal:[0.85587572 0.85631508 0.84914489 0.86262947 0.85778254]


/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

#### GridSearchCV for Logistic Regression

##### Insights: Logistic Regression has a potential of higher prediction accuracy. So we searched the best set of parameters using Grid search.

In [34]:
# Grid search to find the best C, penalty, max_iter, solver, class_weight. 

# H1N1 Model

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2", 'elasticnet', 'none'],
     "max_iter":[100, 1000, 10000, 100000, 1000000], 
      'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga' ],
     "class_weight":['balanced', None]} 

logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=5, scoring='roc_auc')
logreg_cv.fit(X_train_imputed,y_train.h1n1_vaccine)



/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet 

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

...

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet 

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'class_weight': ['balanced', None],
                         'max_iter': [100, 1000, 10000, 100000, 1000000],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='roc_auc')

In [35]:
# Grid search to tune C, penalty, max_iter, solver, class_weight   

# Seasonal Flu

grid_s={"C":np.logspace(-3,3,7), "penalty":["l1","l2", 'elasticnet', 'none'],
     "max_iter":[100, 1000, 10000, 100000, 1000000], 
      'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga' ],
     "class_weight":['balanced', None]} # l1 lasso l2 ridge

logreg=LogisticRegression()
logreg_cv_s=GridSearchCV(logreg,grid_s,cv=5, scoring='roc_auc')
logreg_cv_s.fit(X_train_imputed,y_train.seasonal_vaccine)



/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet 

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

...

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet 

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/emikonaomasa/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none'

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'class_weight': ['balanced', None],
                         'max_iter': [100, 1000, 10000, 100000, 1000000],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='roc_auc')

In [36]:
print('Best parameter for Logistic Regression (H1N1 model):')
print(logreg_cv.best_params_)
print()
print('Best parameter for Logistic Regression (Seasonal Flu model):')
print(logreg_cv_s.best_params_)

Best parameter for Logistic Regression (H1N1 model):
{'C': 1.0, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l1', 'solver': 'saga'}

Best parameter for Logistic Regression (Seasonal Flu model):
{'C': 1.0, 'class_weight': 'balanced', 'max_iter': 10000, 'penalty': 'l1', 'solver': 'saga'}


#### Logistic Regressions with the best set of parameters 
##### Insights:  ROC_AUC scores for both H1N1 model and Seasonal Flu model slightly improved with the best parameters. ROC_AUC score is stable via cross-validation. 

In [37]:
# H1N1 
logreg_h1n1=LogisticRegression(C=1,penalty="l1", class_weight='balanced', max_iter=1000000,
                              solver='saga')


h1n1_1 = cross_val_score(estimator=logreg_h1n1, X=X_train_imputed, y=y_train.h1n1_vaccine, 
                cv=5, scoring='roc_auc')

# Seasonal Flu


log_f_s=LogisticRegression(C=1,penalty="l1", class_weight='balanced', max_iter=10000,
                              solver='saga')

sea_1 = cross_val_score(estimator=log_f_s, X=X_train_imputed, y=y_train.seasonal_vaccine, 
                cv=5, scoring='roc_auc')



print('ROC_AUC for Logistic Regression with best params (H1N1)')
print(h1n1_1)
print()
print('ROC_AUC for Logistic Regression with best params (Seasonal)')
print(sea_1)
print()
print('ROC_AUC for Default Logistic Regression (H1N1)')
print(h1n1)
print()
print('ROC_AUC for Default Logistic Regression (Seasonal)')
print(sea)

ROC_AUC for Logistic Regression with best params (H1N1)
[0.83834717 0.8374085  0.85253863 0.82369232 0.8460361 ]

ROC_AUC for Logistic Regression with best params (Seasonal)
[0.85602188 0.85650111 0.84918733 0.86247773 0.85783659]

ROC_AUC for Default Logistic Regression (H1N1)
[0.83848129 0.83708961 0.85211532 0.82365835 0.84492364]

ROC_AUC for Default Logistic Regression (Seasonal)
[0.85587572 0.85631508 0.84914489 0.86262947 0.85778254]


### LogisticRegression seems to outperform KNN and DecisionTreeClassifiers. Will attempt using the Gradient Boosting Classifier to further optimize predictions

### 3.4 Gradient Boosting Classifier

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

gbc.fit(X)

new_param_grid = {'learning_rate': [0.1, .05],
 'loss': ['exponential'],
 'max_depth': [1,3],
 'max_features': ['log2'],
 'n_estimators': [500,700,900],
 'random_state': [14]}

TypeError: fit() missing 1 required positional argument: 'y'

In [ ]:
gbcgridh1n1 = GridSearchCV(gbc, param_grid= new_param_grid, scoring= 'roc_auc', verbose = 1)

In [ ]:
gbcgridh1n1.fit(X_train_imputed, y_train.h1n1_vaccine)

In [ ]:
gbcgridh1n1.best_params_

final_h1n1_gb_model = GradientBoostingClassifier(loss= 'exponential', learning_rate= .05, random_state = 14,
                                                max_depth=3, max_features='log2', n_estimators=700)
final_h1n1_gb_model.fit(X_train_imputed, y_train['h1n1_vaccine'])

In [ ]:
gbcgridseasonal = GridSearchCV(gbc, param_grid = new_param_grid, scoring= 'roc_auc', verbose = 1)

gbcgridseasonal.fit(X_train_imputed, y_train['seasonal_vaccine'])

gbcgridseasonal.best_params_



In [38]:
seasonalgbcbest = GradientBoostingClassifier(loss= 'exponential', learning_rate= .05, random_state = 14,
                                                max_depth=3, max_features='log2', n_estimators=700)

seasonalgbcbest.fit(X_train_imputed, y_train['seasonal_vaccine'])

NameError: name 'GradientBoostingClassifier' is not defined